# Modeling

### Library & Data Imports

In [15]:
import warnings; warnings.simplefilter('ignore')
import pandas as pd
import numpy as np

from sklearn.linear_model import LassoCV, RidgeCV, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
plt.style.use('ggplot')

In [11]:
data = pd.read_csv('final_data.csv', index_col=0)
data.head()

,name,year_published,min_players,max_players,playing_time,min_age,average_rating,average_weight,category_abstract_strategy,category_action_/_dexterity,...,mechanic_time_track,mechanic_trading,mechanic_trick-taking,mechanic_variable_phase_order,mechanic_variable_player_powers,mechanic_voting,mechanic_worker_placement,category_historical,category_culture,mechanic_movement
0,"Warhammer 40,000: Heroes of Black Reach",2018.0,2.0,2.0,120.0,14.0,7.63636,1.91252,0,0,...,0,0,0,0,0,0,0,0,0,1
1,The Russo-Georgian War of 2008,2010.0,1.0,2.0,90.0,12.0,7.22222,3.00000,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Frher oder Spter,2009.0,2.0,5.0,30.0,8.0,4.50000,1.00000,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Wings Over France,1992.0,1.0,1.0,60.0,10.0,7.50000,3.22220,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Mothra vs. Godzilla,1982.0,2.0,2.0,90.0,10.0,6.98143,1.50000,0,0,...,0,0,0,0,0,0,0,0,1,0


### Setup

In [18]:
y = data['average_rating']
X = data.drop(['name', 'average_rating'], axis=1)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [20]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)